# Download Data from Google Drive

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-03-25

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Download Data from Google Drive
# Author: Timm Nawrocki
# Created on: 2019-03-25
# Usage: Must be executed as a Jupyter Notebook in an Python 2 installation with Google API and oauth2client installed.
# Description: "Download Data from Google Drive" programmatically downloads batches of data from Google Drive. This notebook is intended to download the spectral data exported from Google Earth Engine.
# ---------------------------------------------------------------------------

## 1. Initialize Environment

In [2]:
# Import packages
import os
import io
import httplib2
import logging
from oauth2client.client import OAuth2WebServerFlow
from oauth2client.file import Storage
from apiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from apiclient import errors

# Define Local Drive
drive = 'F:/'

# Define target Google Drive folder
google_folder = '1p_CR7YW3yyvpiorYt_uxuGKlKkmBxLcE' #Sentinel 2
#google_folder = '1ptOXuFUwbnXpmhsW_hUqvI2_BiSRNOLS' #Landsat 8

# Define output directory
output_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS/Data_Input/source_data/imagery/sentinel2')
#output_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS/Data_Input/source_data/imagery/landsat8')

# Define logger
logging.basicConfig(filename='debug.log',level=logging.DEBUG)

# Define client id and secret for Google Drive API
client_id = '655284198639-hh3fg8es5hp9rtmvh4a0t5ulfigns9qq.apps.googleusercontent.com'
client_secret = 'cmelwkw9PFwtUKZ6ZvSQr-IH'

# Define Google Drive authorization information and credentials file
oauth_scope = 'https://www.googleapis.com/auth/drive'
redirect_uri = 'urn:ietf:wg:oauth:2.0:oob'
credentials_file = os.path.join(drive, 'ACCS_Work/Administrative/Credentials/credentials.json')

# Create credentials storage object
storage = Storage(credentials_file)
credentials = storage.get()

In [3]:
# Run through the OAuth flow and retrieve credentials
try:
    flow = OAuth2WebServerFlow(client_id, client_secret, oauth_scope, redirect_uri)
    authorize_url = flow.step1_get_authorize_url()
    print('Go to the following link in your browser: {0}'.format(authorize_url))
    code = raw_input('Enter verification code: ').strip()
    credentials = flow.step2_exchange(code)
    storage.put(credentials)
    print('User successfully authenticated.')
except:
    print('Could not authenticate user. Ensure client id and client secret are correct for Google Drive API.')
    quit()

Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=655284198639-hh3fg8es5hp9rtmvh4a0t5ulfigns9qq.apps.googleusercontent.com&access_type=offline
Enter verification code: 4/GgEQSXkGvqxcNJnUwYICcIF-LRPi5nlXLVJAyQ0bV7ddU5K6jYwkYFU
User successfully authenticated.


In [4]:
# Create an http.client.Http object
http = httplib2.Http()
http = credentials.authorize(http)
drive_service = build('drive', 'v2', http=http)

In [5]:
# Check output directory and create if it does not exist
if os.path.exists(output_directory):
    print('Directory exists.')
else:
    print('Directory does not exist and will be created.')
    os.makedirs(output_directory)

Directory exists.


## 2. Define Functions

In [6]:
# Define a function to create a list of all file IDs within a folder
def list_files(service, folder_id):
    """
    Description: creates a list of files by ID belonging to a Google Drive folder.
    Inputs: service -- Drive API service instance.
            folder_id -- ID of the folder from which to list files.
    """
    # Create empty list to store file IDs
    file_id_list = []
    page_token = None
    # Search folder for files
    while True:
        try:
            param = {}
            if page_token:
                param['pageToken'] = page_token
            children = service.children().list(folderId=folder_id, **param).execute()
            for child in children.get('items', []):
                file_id_list = file_id_list + [child['id']]
            page_token = children.get('nextPageToken')
            if not page_token:
                break
        except errors.HttpError, error:
            print('An error occurred: {0}'.format(error))
            break
    # Return file ID list
    return file_id_list

In [7]:
# Define a function to download data files from Google Drive
def download_file(file_id, output_directory):
    """
    Description: downloads a file from Google Drive by file ID.
    Inputs: file_id -- ID of the file to download.
            output_directory -- directory in which to save the downloaded file.
    """
    # Get file title metadata by file id
    file_meta = drive_service.files().get(fileId=file_id).execute()
    file_title = file_meta['title']
    # Generate download file path
    file_path = os.path.join(output_directory, file_title)
    print('\tSaving {0}...'.format(file_title))
    # Create request, file handler, and downloader
    request = drive_service.files().get_media(fileId=file_id)
    file_handler = io.FileIO(file_path, 'wb')
    downloader = MediaIoBaseDownload(file_handler, request)
    # Download file and report progress
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print('\t\tDownload {0}%...'.format(int(status.progress() * 100)))
    file_handler.close()
    print('\tSave complete.')

## 3. Download Files

In [8]:
# List all files in Google Drive Folder
file_id_list = list_files(drive_service, google_folder)

In [9]:
# Download all files in Google Drive Folder
count = 1
total = len(file_id_list)
for file_id in file_id_list:
    print('Downloading file {0} of {1}...'.format(count, total))
    download_file(file_id, output_directory)
    count += 1

	Saving Sent2_10October_ndwi-0000032768-0000098304.tif...
		Download 15%...
		Download 31%...
		Download 46%...
		Download 62%...
		Download 77%...
		Download 93%...
		Download 100%...
	Save complete.
	Saving Sent2_10October_ndwi-0000032768-0000065536.tif...
		Download 3%...
		Download 7%...
		Download 10%...
		Download 14%...
		Download 18%...
		Download 21%...
		Download 25%...
		Download 29%...
		Download 32%...
		Download 36%...
		Download 40%...
		Download 43%...
		Download 47%...
		Download 51%...
		Download 54%...
		Download 58%...
		Download 61%...
		Download 65%...
		Download 69%...
		Download 72%...
		Download 76%...
		Download 80%...
		Download 83%...
		Download 87%...
		Download 91%...
		Download 94%...
		Download 98%...
		Download 100%...
	Save complete.
	Saving Sent2_10October_ndwi-0000032768-0000032768.tif...
		Download 3%...
		Download 6%...
		Download 9%...
		Download 13%...
		Download 16%...
		Download 19%...
		Download 22%...
		Download 26%...
		Download 29%...
		Dow

	Saving Sent2_10October_ndvi-0000000000-0000032768.tif...
		Download 2%...
		Download 5%...
		Download 8%...
		Download 10%...
		Download 13%...
		Download 16%...
		Download 18%...
		Download 21%...
		Download 24%...
		Download 26%...
		Download 29%...
		Download 32%...
		Download 34%...
		Download 37%...
		Download 40%...
		Download 42%...
		Download 45%...
		Download 48%...
		Download 50%...
		Download 53%...
		Download 56%...
		Download 58%...
		Download 61%...
		Download 64%...
		Download 66%...
		Download 69%...
		Download 72%...
		Download 74%...
		Download 77%...
		Download 80%...
		Download 82%...
		Download 85%...
		Download 88%...
		Download 91%...
		Download 93%...
		Download 96%...
		Download 99%...
		Download 100%...
	Save complete.
	Saving Sent2_10October_ndvi-0000000000-0000000000.tif...
		Download 2%...
		Download 5%...
		Download 7%...
		Download 10%...
		Download 12%...
		Download 15%...
		Download 18%...
		Download 20%...
		Download 23%...
		Download 25%...
		Downloa

		Download 97%...
		Download 100%...
	Save complete.
	Saving Sent2_10October_evi2-0000000000-0000069888.tif...
		Download 2%...
		Download 4%...
		Download 7%...
		Download 9%...
		Download 11%...
		Download 14%...
		Download 16%...
		Download 18%...
		Download 21%...
		Download 23%...
		Download 25%...
		Download 28%...
		Download 30%...
		Download 33%...
		Download 35%...
		Download 37%...
		Download 40%...
		Download 42%...
		Download 44%...
		Download 47%...
		Download 49%...
		Download 51%...
		Download 54%...
		Download 56%...
		Download 59%...
		Download 61%...
		Download 63%...
		Download 66%...
		Download 68%...
		Download 70%...
		Download 73%...
		Download 75%...
		Download 77%...
		Download 80%...
		Download 82%...
		Download 84%...
		Download 87%...
		Download 89%...
		Download 92%...
		Download 94%...
		Download 96%...
		Download 99%...
		Download 100%...
	Save complete.
	Saving Sent2_10October_evi2-0000000000-0000046592.tif...
		Download 2%...
		Download 4%...
		Download

		Download 41%...
		Download 45%...
		Download 50%...
		Download 55%...
		Download 59%...
		Download 64%...
		Download 68%...
		Download 73%...
		Download 78%...
		Download 82%...
		Download 87%...
		Download 91%...
		Download 96%...
		Download 100%...
	Save complete.
	Saving Sent2_10October_8_nearInfrared-0000032768-0000032768.tif...
		Download 4%...
		Download 8%...
		Download 12%...
		Download 16%...
		Download 20%...
		Download 24%...
		Download 28%...
		Download 32%...
		Download 36%...
		Download 40%...
		Download 44%...
		Download 48%...
		Download 52%...
		Download 56%...
		Download 61%...
		Download 65%...
		Download 69%...
		Download 73%...
		Download 77%...
		Download 81%...
		Download 85%...
		Download 89%...
		Download 93%...
		Download 97%...
		Download 100%...
	Save complete.
	Saving Sent2_10October_8_nearInfrared-0000032768-0000000000.tif...
		Download 5%...
		Download 10%...
		Download 15%...
		Download 21%...
		Download 26%...
		Download 31%...
		Download 36%...
		Dow

		Download 41%...
		Download 62%...
		Download 82%...
		Download 100%...
	Save complete.
	Saving Sent2_10October_4_red-0000032768-0000065536.tif...
		Download 4%...
		Download 9%...
		Download 14%...
		Download 19%...
		Download 24%...
		Download 29%...
		Download 34%...
		Download 39%...
		Download 44%...
		Download 48%...
		Download 53%...
		Download 58%...
		Download 63%...
		Download 68%...
		Download 73%...
		Download 78%...
		Download 83%...
		Download 88%...
		Download 92%...
		Download 97%...
		Download 100%...
	Save complete.
	Saving Sent2_10October_4_red-0000032768-0000032768.tif...
		Download 4%...
		Download 8%...
		Download 12%...
		Download 17%...
		Download 21%...
		Download 25%...
		Download 30%...
		Download 34%...
		Download 38%...
		Download 43%...
		Download 47%...
		Download 51%...
		Download 56%...
		Download 60%...
		Download 64%...
		Download 68%...
		Download 73%...
		Download 77%...
		Download 81%...
		Download 86%...
		Download 90%...
		Download 94%...
		Down

...

		Download 2%...
		Download 4%...
		Download 7%...
		Download 9%...
		Download 12%...
		Download 14%...
		Download 17%...
		Download 19%...
		Download 22%...
		Download 24%...
		Download 27%...
		Download 29%...
		Download 32%...
		Download 34%...
		Download 37%...
		Download 39%...
		Download 42%...
		Download 44%...
		Download 46%...
		Download 49%...
		Download 51%...
		Download 54%...
		Download 56%...
		Download 59%...
		Download 61%...
		Download 64%...
		Download 66%...
		Download 69%...
		Download 71%...
		Download 74%...
		Download 76%...
		Download 79%...
		Download 81%...
		Download 84%...
		Download 86%...
		Download 88%...
		Download 91%...
		Download 93%...
		Download 96%...
		Download 98%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_evi2-0000046592-0000093184.tif...
		Download 64%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_evi2-0000046592-0000069888.tif...
		Download 5%...
		Download 11%...
		Download 16%...
		Download 22%...
		Download 27%...

		Download 94%...
		Download 97%...
		Download 99%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_evi2-0000000000-0000046592.tif...
		Download 2%...
		Download 4%...
		Download 7%...
		Download 9%...
		Download 12%...
		Download 14%...
		Download 17%...
		Download 19%...
		Download 22%...
		Download 24%...
		Download 27%...
		Download 29%...
		Download 32%...
		Download 34%...
		Download 37%...
		Download 39%...
		Download 42%...
		Download 44%...
		Download 47%...
		Download 49%...
		Download 52%...
		Download 54%...
		Download 57%...
		Download 59%...
		Download 62%...
		Download 64%...
		Download 67%...
		Download 69%...
		Download 72%...
		Download 74%...
		Download 77%...
		Download 79%...
		Download 82%...
		Download 84%...
		Download 87%...
		Download 89%...
		Download 92%...
		Download 94%...
		Download 97%...
		Download 99%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_evi2-0000000000-0000023296.tif...
		Download 2%...
		Download 4%...
		Download 7%...
	

		Download 5%...
		Download 11%...
		Download 16%...
		Download 22%...
		Download 28%...
		Download 33%...
		Download 39%...
		Download 45%...
		Download 50%...
		Download 56%...
		Download 61%...
		Download 67%...
		Download 73%...
		Download 78%...
		Download 84%...
		Download 90%...
		Download 95%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_12_shortInfrared2-0000000000-0000000000.tif...
		Download 3%...
		Download 7%...
		Download 11%...
		Download 15%...
		Download 19%...
		Download 23%...
		Download 27%...
		Download 31%...
		Download 35%...
		Download 38%...
		Download 42%...
		Download 46%...
		Download 50%...
		Download 54%...
		Download 58%...
		Download 62%...
		Download 66%...
		Download 70%...
		Download 74%...
		Download 77%...
		Download 81%...
		Download 85%...
		Download 89%...
		Download 93%...
		Download 97%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_8a_redEdge4-0000000000-0000032768.tif...
		Download 5%...
		Download 10%...
		Download 15%

		Download 10%...
		Download 15%...
		Download 20%...
		Download 25%...
		Download 30%...
		Download 35%...
		Download 40%...
		Download 45%...
		Download 50%...
		Download 55%...
		Download 60%...
		Download 65%...
		Download 70%...
		Download 75%...
		Download 80%...
		Download 85%...
		Download 90%...
		Download 95%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_3_green-0000032768-0000032768.tif...
		Download 4%...
		Download 8%...
		Download 13%...
		Download 17%...
		Download 22%...
		Download 26%...
		Download 31%...
		Download 35%...
		Download 40%...
		Download 44%...
		Download 48%...
		Download 53%...
		Download 57%...
		Download 62%...
		Download 66%...
		Download 71%...
		Download 75%...
		Download 80%...
		Download 84%...
		Download 88%...
		Download 93%...
		Download 97%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_3_green-0000032768-0000000000.tif...
		Download 5%...
		Download 11%...
		Download 17%...
		Download 23%...
		Download 29%...
		Downloa

		Download 61%...
		Download 66%...
		Download 72%...
		Download 77%...
		Download 83%...
		Download 88%...
		Download 94%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_5_redEdge1-0000000000-0000000000.tif...
		Download 3%...
		Download 7%...
		Download 11%...
		Download 15%...
		Download 19%...
		Download 23%...
		Download 27%...
		Download 31%...
		Download 34%...
		Download 38%...
		Download 42%...
		Download 46%...
		Download 50%...
		Download 54%...
		Download 58%...
		Download 62%...
		Download 65%...
		Download 69%...
		Download 73%...
		Download 77%...
		Download 81%...
		Download 85%...
		Download 89%...
		Download 93%...
		Download 97%...
		Download 100%...
	Save complete.
	Saving Sent2_05May_1_ultraBlue.tif...
		Download 23%...
		Download 46%...
		Download 70%...
		Download 93%...
		Download 100%...
	Save complete.
